In [1]:
import requests
import gtfs_realtime_pb2  # Ensure this file is generated

# URL for MTA's GTFS-Realtime data
MTA_FEED_URL = 'https://api-endpoint.mta.info/Dataservice/mtagtfsfeeds/nyct%2Fgtfs-ace'


In [2]:
def fetch_mta_alerts():
    # Initialize the GTFS-Realtime FeedMessage
    feed = gtfs_realtime_pb2.FeedMessage()
    
    try:
        # Fetch GTFS-Realtime data from the MTA
        response = requests.get(MTA_FEED_URL)
        response.raise_for_status()
        feed.ParseFromString(response.content)
    except requests.RequestException as e:
        print("Failed to fetch MTA data:", e)
        return []

    # Collect all alerts
    alerts = []
    for entity in feed.entity:
        if entity.HasField('alert'):
            alert = entity.alert
            alert_text = alert.header_text.translation[0].text
            alerts.append(alert_text)
    return alerts


In [3]:
def filter_alerts_by_station(alerts, line, station):
    # Filter alerts for the specified line and station
    relevant_alerts = [alert for alert in alerts if line.lower() in alert.lower() and station.lower() in alert.lower()]
    return relevant_alerts


In [4]:
def main():
    # Get user input
    line = input("Enter subway line (e.g., A, C, E): ").strip().upper()
    station = input("Enter station name (e.g., 34th St): ").strip().title()
    
    # Fetch all MTA alerts
    alerts = fetch_mta_alerts()
    
    if not alerts:
        print("No alerts found.")
        return
    
    # Filter alerts by line and station
    relevant_alerts = filter_alerts_by_station(alerts, line, station)
    
    # Display the filtered alerts
    if relevant_alerts:
        print(f"\nService Alerts for {line} line at {station}:")
        for alert in relevant_alerts:
            print("-", alert)
    else:
        print(f"\nNo current service alerts for {line} line at {station}.")


In [ ]:
main()

